In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import collections
train_set_num, test_set_num = 500,500

In [2]:
def concate_files(file1,file2,final_file):
      data = data2 = ""
      with open(file1, 'r', encoding='utf-8') as fp:  # Reading data from file1
            sentences = fp.readlines()
            for sentence in sentences[:train_set_num]:
                  data += sentence
      with open(file2, 'r', encoding='utf-8') as fp:  # Reading data from file2
            sentences = fp.readlines()
            for sentence in sentences[:test_set_num]:
                  data2 += sentence
      data += data2  # To add the data of file2
      with open (final_file, 'w', encoding='utf-8') as fp:
            fp.write(data)
concate_files('data/total/pos.txt','data/total/neg.txt','data/total/labeled_text.txt')

In [3]:
data = pd.read_csv('data/total/labeled_text.txt', header = None, delimiter='    ',encoding='utf-8',names=['Label', 'Text'])
data.head()

d:\Downloads\python3.7\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,Label,Text
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"


In [4]:
data.Label.unique()

array([ 1, -1], dtype=int64)

In [5]:
data.shape

(1000, 2)

In [6]:
stopword_ls = []
def getStopWord():
    with open('lib/stopwords_utf8.txt', 'r',encoding='UTF-8') as file:
        for line in file:
            stopword_ls.append(line.split('\n')[0])
getStopWord()

def isStopWord(word):
    for i in range(len(stopword_ls)):
        if word == stopword_ls[i]:
            return True
    return False

In [7]:
import jieba
from snownlp import SnowNLP
import random

def jieba_segment(text):
    word_ls = []
    tmp_ls = list(jieba.cut(text, cut_all=False))   #segmentation
    for i in range(len(tmp_ls)):
        if not isStopWord(tmp_ls[i]):
            word_ls.append(tmp_ls[i]) 
    return word_ls

def snow_segment(text):
    word_ls = []
    tmp_ls = SnowNLP(text).words  #segmentation  
    for i in range(len(tmp_ls)):
        if not isStopWord(tmp_ls[i]):
            word_ls.append(tmp_ls[i]) 
    return word_ls

random.shuffle(data['Text'])
#data['Segmented_Text'] = data['Text'].apply(lambda x: jieba_segment(x))
data['Segmented_Text'] = data['Text'].apply(lambda x: snow_segment(x))
data.head()

d:\Downloads\python3.7\lib\random.py:278: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i], x[j] = x[j], x[i]


,Label,Text,Segmented_Text
0,1,"这家酒店的设施真的非常不错.地理位置也方便,大堂气派.由于没有我预订的那种房型,到前台的时候...","[家, 酒店, 设施, 真, 不错, 地理, 位置, 大堂, 气派, 预订, 那种, 房, ..."
1,1,招待所而已吧，唯一可取之处就是交通还行，三环边上。,"[招待所, 唯一, 取之处, 交通, 还行, 环, 边上]"
2,1,看了前面的点评，感觉此酒店应该不错，所以这次去慈溪定了国脉酒店。结果大失所望。酒店位置不错，...,"[点评, 感觉, 酒店, 不错, 慈溪, 国脉酒店, 大失所望, 酒店, 位置, 不错, ，..."
3,1,"先说好的:地段在闹市区中心,房间也还可以,价格比较便宜,含在188房价里的早餐也可以.不好的...","[先, 说, 地段, 闹市区, 中心, 房间, 价格, 便宜, 含, 188, 房价, 里,..."
4,1,"住过的最差挂牌五星酒店!!!所订的行政高级江景房面对富春江,不过酒店和江中间有个中学隔着,酒...","[住, 差, 挂牌, 五星酒店, !!!, 所订, 行政, 高级, 江, 景房, 面对, 富..."


In [ ]:
#Assign Label to each sentence
PosNegLabel = []
for label in data.Label:
    if label == -1:        #negative
        PosNegLabel.append(0)
    elif label == 1:      #positive
        PosNegLabel.append(1)
data['Label']= PosNegLabel
data = data[['Segmented_Text','Label']]
data.head()

In [8]:
data_train, data_test = train_test_split(data, test_size=0.20)

In [9]:
all_training_words = [word for tokens in data_train["Segmented_Text"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["Segmented_Text"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("TRAIN DATASET | Total Words:{} | Total Vocabulary:{} | Max Sentence Length:{}".format(len(all_training_words),len(TRAINING_VOCAB),max(training_sentence_lengths)))

all_test_words = [word for tokens in data_test["Segmented_Text"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test["Segmented_Text"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("TEST  DATASET | Total Words:{}  | Total Vocabulary:{} | Max Sentence Length:{}" .format(len(all_test_words),len(TEST_VOCAB),max(test_sentence_lengths)))

TRAIN DATASET | Total Words:32762 | Total Vocabulary:6051 | Max Sentence Length:578
TEST  DATASET | Total Words:8458  | Total Vocabulary:2738 | Max Sentence Length:578


In [10]:
#Setting up Vocabulary Dict
vocab = collections.Counter()
for twit in data_train.Segmented_Text:
    for word in twit:
        vocab[word] += 1
vocab_len = len(vocab)  
print(vocab_len,'Vocabularies')
print('Top 10 Vocabularies:')
vocab_ls = list(vocab)
vocab.most_common(10)

6051 Vocabularies
Top 10 Vocabularies:


[('酒店', 915),
 ('房间', 756),
 ('住', 501),
 ('服务', 444),
 ('不错', 337),
 ('说', 275),
 ('差', 262),
 ('太', 228),
 ('房', 222),
 ('宾馆', 214)]

In [11]:
length = len(data_train.Segmented_Text)
word2vec_train = np.zeros([length,vocab_len])
k = 0
for twit in data_train.Segmented_Text:
      k += 1
      if k == length:
            end_val = ' | '
      else:
            end_val = '\r'
      print('Getting Train Dataset Word2vec: {}/{}'.format(k,length),end=end_val)
      for i in range(len(twit)):
            for j in range(vocab_len):
                  if twit[i] == vocab_ls[j]:
                        #word2vec_train[k-1][j] = 1*vocab[vocab_ls[j]]/vocab_len
                        word2vec_train[k-1][j] = 1
print('Vector Shape:',word2vec_train.shape)

length = len(data_test.Segmented_Text)
word2vec_test = np.zeros([length,vocab_len])
k = 0
for twit in data_test.Segmented_Text:
      k += 1
      if k == length:
            end_val = ' | '
      else:
            end_val = '\r'
      print('Getting Test Dataset Word2vec: {}/{}'.format(k,length),end=end_val)
      for i in range(len(twit)):
            for j in range(vocab_len):
                  if twit[i] == vocab_ls[j]:
                        #word2vec_test[k-1][j] = 1*vocab[vocab_ls[j]]/vocab_len
                        word2vec_test[k-1][j] = 1
print('Vector Shape:', word2vec_test.shape)


Getting Train Dataset Word2vec: 800/800 | Vector Shape: (800, 6051)
Getting Test Dataset Word2vec: 200/200 | Vector Shape: (200, 6051)


In [33]:
def preprocess(data_vector):  #data:list | vocab:torchtext.legacy.vocab.Vocab
    max_l = 500  
    def set_same_length(x):   #将每条评论通过截断或者补0，使得长度变成500
        return x[:max_l] if len(x) > max_l else x + [0] * (max_l - len(x))  
    features = [set_same_length(words) for words in data_vector]
    return np.array(features)

In [56]:
X_train, X_test = preprocess(word2vec_train), preprocess(word2vec_test)
y_train, y_test = np.array(data_train['Label']), np.array(data_test['Label'])
X_train.shape[1]

500

In [48]:
class LogisticRegression_:
    
    # defining parameters such as learning rate, number ot iterations, whether to include intercept, 
    # and verbose which says whether to print anything or not like, loss etc.
    def __init__(self, learning_rate=0.01, num_iterations=50000, fit_intercept=True, verbose=False):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.fit_intercept = fit_intercept
        self.verbose = verbose
    
    # function to define the Incercept value.
    def __b_intercept(self, X):
        # initially we set it as all 1's
        intercept = np.ones((X.shape[0], 1))
        # then we concatinate them to the value of X, we don't add we just append them at the end.
        return np.concatenate((intercept, X), axis=1)
    
    def __sigmoid_function(self, z):
        # this is our actual sigmoid function which predicts our yp
        return 1 / (1 + np.exp(-z))
    
    def __loss(self, yp, y):
        # this is the loss function which we use to minimize the error of our model
        return (-y * np.log(yp) - (1 - y) * np.log(1 - yp)).mean()

    # this is the function which trains our model.
    def fit(self, X, y):
        
        # as said if we want our intercept term to be added we use fit_intercept=True
        if self.fit_intercept:
            X = self.__b_intercept(X)
        
        # weights initialization of our Normal Vector, initially we set it to 0, then we learn it eventually
        self.W = np.zeros(X.shape[1])
        
        # this for loop runs for the number of iterations provided
        for i in range(self.num_iterations):
            
            # this is our W * Xi
            z = np.dot(X, self.W)
            
            # this is where we predict the values of Y based on W and Xi
            yp = self.__sigmoid_function(z)
            
            # this is where the gradient is calculated form the error generated by our model
            gradient = np.dot(X.T, (yp - y)) / y.size
            
            # this is where we update our values of W, so that we can use the new values for the next iteration
            self.W -= self.learning_rate * gradient
            
            # this is our new W * Xi
            z = np.dot(X, self.W)
            yp = self.__sigmoid_function(z)

            # this is where the loss is calculated
            loss = self.__loss(yp, y)
            
            # as mentioned above if we want to print somehting we use verbose, so if verbose=True then our loss get printed
            if(self.verbose ==True and i % 10000 == 0):
                print(f'loss: {loss} \t')

    # this is where we predict the probability values based on out generated W values out of all those iterations.
    def predict_prob(self, X):
        # as said if we want our intercept term to be added we use fit_intercept=True
        if self.fit_intercept:
            X = self.__b_intercept(X)
        
        # this is the final prediction that is generated based on the values learned.
        return self.__sigmoid_function(np.dot(X, self.W))
    
    # this is where we predict the actual values 0 or 1 using round. anything less than 0.5 = 0 or more than 0.5 is 1
    def predict(self, X):
        return self.predict_prob(X).round()

In [49]:
model = LogisticRegression_(learning_rate=0.1, num_iterations=30000)

In [50]:
%%time
model.fit(X_train, y_train)

C:\Users\User\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:20: RuntimeWarning: overflow encountered in exp
C:\Users\User\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in log
d:\Downloads\python3.7\lib\site-packages\numpy\core\_methods.py:179: RuntimeWarning: invalid value encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)


Wall time: 13.9 s


In [51]:
preds = model.predict(X_test)
(preds == y_test).mean()

C:\Users\User\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:20: RuntimeWarning: overflow encountered in exp


0.0

In [52]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00      90.0
         0.0       0.00      0.00      0.00       0.0
         1.0       0.00      0.00      0.00     110.0

    accuracy                           0.00     200.0
   macro avg       0.00      0.00      0.00     200.0
weighted avg       0.00      0.00      0.00     200.0



C:\Users\User\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:20: RuntimeWarning: overflow encountered in exp
d:\Downloads\python3.7\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Downloads\python3.7\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Downloads\python3.7\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beha

In [53]:
print('Importing Logistic Regression Model')
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
log = LogisticRegression().fit(X_train,y_train)
# regression coefficients
print('Coefficients:', log.coef_)
print('Train Accuracy Score: {}'.format(log.score(X_train, y_train)))
print('Test Accuracy Score: {}'.format(log.score(X_test, y_test)))

Importing Logistic Regression Model
Coefficients: [[-0.17568586  0.49225183 -0.45599671  0.30097369 -0.1638631   0.33767669
   0.0407593  -0.52323747  0.21060834 -0.33735663 -0.73626777  0.27084881
  -0.24751186 -0.15148616  0.37974569  0.54458144  0.33280253 -0.11882221
   0.05910231 -0.76536397 -0.17745679 -0.15280842  0.22242455 -0.14298472
   0.07386455  0.19753083 -0.14322125 -0.43348472 -0.15148616 -0.15148616
  -0.15148616  0.18651403 -0.07015757  0.12975248  0.02814355 -0.14129276
  -0.24601196 -0.19804222 -0.46968628  0.38748435  0.42001384 -0.22573736
   0.29810968  0.31186421 -0.00196985  0.43901861  0.12028254  0.31186421
   0.31186421 -0.80117665 -0.1848918  -0.6529479   0.31186421  0.24862942
   0.01586296 -0.17822889 -0.64089157 -0.4341945   0.53271523  0.21434434
   0.01957897  0.30337923  0.04583407  0.79920745 -0.15405928 -0.26858628
   0.16459842 -0.02807628  0.65799871  0.03757108  0.12558826 -0.56341648
   0.4156013  -0.29976014 -0.51054637  0.34405178  0.22727567 

In [54]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [55]:
import math
def plot_distribution(vocabulary):
    hist, edges = np.histogram(list(map(lambda x:math.log(x[1]),vocabulary.most_common())), density=True, bins=500)
    p = figure(tools="pan,wheel_zoom,reset,save",
               toolbar_location="above",
               title="Word distribution accross all sentences")
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555", )
    show(p)
plot_distribution(vocab)